In [18]:
from clickhouse_driver import Client
from datetime import date, datetime, timedelta
import random
import pandas
from kafka import KafkaProducer
import json
import random
import os

def json_serial(obj):
    """JSON serializer for objects not serializable by default json code"""

    if isinstance(obj, (datetime, date)):
        return obj.isoformat()
    raise TypeError ("Type %s not serializable" % type(obj))


In [19]:
CLICKHOUSE_HOST=os.getenv("CLICKHOUSE_HOST")
CLICKHOUSE_USERNAME=os.getenv("CLICKHOUSE_USERNAME")
CLICKHOUSE_PASSWORD=os.getenv("CLICKHOUSE_PASSWORD")
client = Client(CLICKHOUSE_HOST, user=CLICKHOUSE_USERNAME, password=CLICKHOUSE_PASSWORD)

KAFKA_BOOTSTRAP_HOST = os.getenv("KAFKA_BOOTSTRAP_HOST")
producer = KafkaProducer(bootstrap_servers=KAFKA_BOOTSTRAP_HOST)

In [20]:
%%time

for station_id in range(100):
    for i in range(10):
        msg = {
            "station_id": station_id,
            "measurement_moment": datetime.now() + timedelta(seconds=i),
            "temperature_ambient": random.uniform(-10,30),
            "temperature_ground": random.uniform(-10,30),
            "humidity": random.uniform(0,1),
            "pressure": random.uniform(980,1030),
            "wind_speed": random.uniform(0,30),
            "precipitation": random.uniform(0,20),
            "irradiance": random.uniform(0,1000) 
        }
        
        producer.send('weather-station-observation', json.dumps(msg, default=json_serial).encode(), key=str(station_id+1).encode())

CPU times: user 45.2 ms, sys: 16 ms, total: 61.1 ms
Wall time: 42.5 ms


Note: It can take up to 7500 ms for data to show up in tables

In [25]:
data, cols = client.execute("select * from weatherStationObservation", with_column_types=True)
pandas.DataFrame(data, columns=[col_name for col_name, _ in cols])

,station_id,measurement_moment,temperature_ambient,temperature_ground,humidity,pressure,wind_speed,precipitation,irradiance
